# GR5241 Project4 
## Doubly Robust Estimation + L1 penalized logistic regression

### L1 penalized logistic regression to calculate propensity score

In [70]:
# Import required libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


#### Data Preprocessing

In [71]:
%pwd 
%cd /Users/duansiyu/Downloads

[Errno 2] No such file or directory: '/Users/duansiyu/Downloads'
/Users/lingjiazhang/Desktop/Columbia University Academic/Fall 2020/GR5243/Project 4/Fall2020-Project4-group-2/doc


In [72]:
start_time = time.time()
# Load data

highDim = pd.read_csv(r'/Users/lingjiazhang/Downloads/data/highDim_dataset.csv')
lowDim = pd.read_csv(r'/Users/lingjiazhang/Downloads/data/lowDim_dataset.csv')
#highDim = pd.read_csv(r'highDim_dataset.csv')
#lowDim = pd.read_csv(r'lowDim_dataset.csv')


# View high dimensional data
highDim.head()

,Y,A,V1,V2,V3,V4,V5,V6,V7,V8,...,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185
0,-11.682472,1,0,1,2,16,3,-1,13,-0.13,...,5,7,8,6,8,-1,-1,-1,-1,-1
1,-13.176546,0,1,1,12,14,14,14,13,0.24,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-2.195401,1,0,1,21,22,10,10,14,0.27,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-0.005454,1,1,1,9,20,11,2,10,0.09,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
4,-1.987538,1,1,1,7,16,16,11,6,0.15,...,70,70,80,70,80,-10,-10,-10,-10,-10


In [73]:
# Create X from the features for L1 Logistic regression 
X_high = highDim.drop(['A','Y'], axis = 1)
X_low = lowDim.drop(['A','Y'], axis = 1)

# Create y from output
y_high = highDim[['A']]
y_low = lowDim[['A']]

#### Split The Data Into Training And Test Sets

In [74]:
# Split The Data Into Training And Test Sets
X_train_high, X_test_high, y_train_high, y_test_high = train_test_split(X_high, y_high, test_size=0.25, random_state=0)
X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(X_low, y_low, test_size=0.25, random_state=0)

#### Standardize Features

Because the regularization penalty is comprised of the sum of the absolute value of the coefficients, we need to scale the data so the coefficients are all based on the same scale.

In [75]:
# Create a scaler object
sc = StandardScaler()

# High Dimensional data
# Fit the scaler to the training data and transform
X_train_high_std = sc.fit_transform(X_train_high)

# Apply the scaler to the test data
X_test_high_std = sc.transform(X_test_high)

# Low Dimensional data
# Fit the scaler to the training data and transform
X_train_low_std = sc.fit_transform(X_train_low)

# Apply the scaler to the test data
X_test_low_std = sc.transform(X_test_low)

#### Fit logistic regression with a L1 penalty & Tunning parameters

In [76]:
C = [.06, .05, .04, .03, .02, .01, 0.008, 0.005, 0.001]

# High Dimensional data
for c in C:
    clf = LogisticRegression(penalty='l1', C = c, solver = 'liblinear')
    clf.fit(X_train_high, y_train_high)
    print('C:', c)
    print('Coefficient of each feature:', clf.coef_)
    print('Training accuracy:', clf.score(X_train_high_std, y_train_high))
    print('Test accuracy:', clf.score(X_test_high_std, y_test_high))
    print('')



/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C: 0.06
Coefficient of each feature: [[ 0.00000000e+00  0.00000000e+00  1.56490146e-02  7.26418891e-03
   1.63071884e-02 -2.36729132e-02  1.70712206e-03  0.00000000e+00
   0.00000000e+00 -2.97019788e-02 -2.16555404e-02  2.79088661e-03
   6.31367147e-03  6.33906966e-03  7.37717304e-03 -2.81513111e-03
  -7.36767744e-05  0.00000000e+00  5.60887275e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -3.42282194e-02  0.00000000e+00  1.15626950e-02
   8.35792260e-04  0.00000000e+00  3.44550143e-04 -5.16480563e-03
   1.51409929e-05  0.00000000e+00  1.18362032e-02  1.02040827e-02
  -6.76541637e-04 -1.10362736e-03  0.00000000e+00  3.45543346e-05
  -8.67033557e-03  2.29190399e-02 -3.52873201e-02  5.12346080e-02
   5.36883234e-02  0.00000000e+00  0.00000000e+00 -1.74551801e-02
   0.00000000e+00  1.63600824e-02 -4.96826470e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.11103835e-03  0.00000000e+00
  -1.39264334e-03 -6.16256082e-02  0.00

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C: 0.05
Coefficient of each feature: [[ 0.00000000e+00  0.00000000e+00  1.17780066e-02  7.69220953e-03
   1.52230116e-02 -2.17617163e-02  1.49865030e-03  0.00000000e+00
   0.00000000e+00 -2.84791794e-02 -1.31863828e-02  2.40370577e-03
   5.81501818e-03  5.92628058e-03  6.85881359e-03 -2.76506865e-03
  -2.18595493e-04  0.00000000e+00  2.15143638e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -3.10174184e-02  0.00000000e+00  9.17564402e-03
   8.33306894e-04  0.00000000e+00  3.63813762e-04 -4.80306483e-03
   1.42927573e-05  0.00000000e+00  1.10244071e-02  8.94583614e-03
  -6.45445263e-04 -1.10691859e-03  0.00000000e+00  1.83624777e-05
  -7.32222975e-03  1.92907269e-02 -3.27198643e-02  4.98168084e-02
   4.79896068e-02  0.00000000e+00  0.00000000e+00 -1.41263544e-02
   0.00000000e+00  1.51920797e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -6.00956661e-02  0.00

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C: 0.03
Coefficient of each feature: [[ 0.00000000e+00  0.00000000e+00  1.69265686e-03  8.79171119e-03
   1.09142254e-02 -1.63833145e-02  7.79983662e-04  0.00000000e+00
   0.00000000e+00 -2.40294217e-02  0.00000000e+00  1.79308814e-03
   4.85242058e-03  5.21302225e-03  5.49623547e-03 -2.00483302e-03
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -2.17054095e-02  0.00000000e+00  3.69181767e-03
   8.82759521e-04  0.00000000e+00  4.80902082e-04 -3.92020399e-03
   1.24802585e-05  0.00000000e+00  7.66409373e-03  3.58667469e-03
  -5.27217236e-04 -1.04588535e-03  0.00000000e+00  1.97711425e-05
  -1.65037707e-03  3.06232166e-03 -1.65060035e-02  4.56166919e-02
   4.07496719e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  8.83295756e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -5.21635657e-02  0.00

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C: 0.008
Coefficient of each feature: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68492365e-03
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -4.70140492e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  3.21809541e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   7.87345118e-04  0.00000000e+00  7.40713571e-05 -1.04022500e-03
   4.62497936e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.20333000e-04 -7.26050965e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -1.58396005e-03  0.0

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [77]:
C = [1, 0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.5, 0.3, 0.2, 0.1]

# Low Dimensional data
for c in C:
    clf = LogisticRegression(penalty='l1', C = c, solver = 'liblinear')
    clf.fit(X_train_low, y_train_low)
    print('C:', c)
    print('Coefficient of each feature:', clf.coef_)
    print('Training accuracy:', clf.score(X_train_low_std, y_train_low))
    print('Test accuracy:', clf.score(X_test_low_std, y_test_low))
    print('')   

C: 1
Coefficient of each feature: [[ 0.34417025  0.43237849  0.73102021 -0.15445061  0.          0.133062
   0.          0.2622922   0.          0.3335332  -0.05791753  0.
   0.04014482 -0.50944865  0.14781836  0.          0.2449429   0.10428947
   0.          0.          0.00274724  0.41842225]]
Training accuracy: 0.7837078651685393
Test accuracy: 0.7226890756302521

C: 0.95
Coefficient of each feature: [[ 3.41394018e-01  4.19111040e-01  7.21248354e-01 -1.47476100e-01
   0.00000000e+00  1.30750952e-01  0.00000000e+00  2.60293289e-01
   0.00000000e+00  3.21964663e-01 -4.59256640e-02  0.00000000e+00
   3.64696995e-02 -5.04461556e-01  1.46541072e-01  0.00000000e+00
   2.44912134e-01  1.03650831e-01  0.00000000e+00  0.00000000e+00
   2.89680894e-04  4.16470009e-01]]
Training accuracy: 0.7837078651685393
Test accuracy: 0.7142857142857143

C: 0.9
Coefficient of each feature: [[ 0.33874808  0.40525787  0.71105139 -0.13972402  0.          0.12832607
   0.          0.2579722   0.          0.30

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

#### Best model for high dimensional data

In [78]:
# Best: C = 0.06
clf_high = LogisticRegression(penalty='l1', C = 0.06, solver = 'liblinear')

#### Best model for low dimensional data

In [79]:
# Best: C = 1
clf_low = LogisticRegression(penalty='l1', C = 1, solver = 'liblinear')

#### Calculate propensity scores

In [80]:
# High dimensional propensity score
clf_high.fit(X_high, y_high.values.ravel())
ps_high=clf_high.predict_proba(X_high)[:, 1]

In [81]:
# Low dimensional propensity score
clf_low.fit(X_low, y_low.values.ravel())
ps_low=clf_low.predict_proba(X_low)[:, 1]

#### L1 logistic regression time

In [82]:
l1_time = time.time() - start_time
print("run time: %s seconds" % (l1_time))

run time: 16.729211807250977 seconds


### Doubly Robust Estimation  Algorithm for ATE

#### Add propensity scores to the data frame

In [83]:
# set time
start_time_dre = time.time()

full_high_dim= highDim.copy()
full_low_dim = lowDim.copy()

In [84]:
full_high_dim['PS']=pd.Series(ps_high, index=full_high_dim.index)
full_low_dim['PS']=pd.Series(ps_low, index=full_low_dim.index)

#### Calculate ATE 

##### Low dimensional Data

In [85]:
# deviding the low dimensional data into treated and control groups

lowDim_treated = lowDim[lowDim['A'] == 1]
lowDim_treated = lowDim_treated.reset_index(drop = True)

lowDim_control = lowDim[lowDim['A'] == 0]
lowDim_control = lowDim_control.reset_index(drop = True)

#Fit a regression model to get the estimation of y given T=1 and X 
#Treated group
X1_low_treated = lowDim_treated.drop(['Y'], axis = 1)
y_low_treated = lowDim_treated['Y']

lr_low_treated = LinearRegression().fit(X1_low_treated, y_low_treated)

# Control group
# Fit a regression model to get the estimation of y given T=0and X 
X1_low_control = lowDim_control.drop(['Y'], axis = 1)
y_low_control = lowDim_control['Y']

lr_low_control = LinearRegression().fit(X1_low_control, y_low_control)

##### High dimensional Data

In [86]:
# deviding the high dimensional data into treated and control groups
highDim_treated = highDim[highDim['A'] == 1]
highDim_treated = highDim_treated.reset_index(drop = True)

highDim_control = highDim[highDim['A'] == 0]
highDim_control = highDim_control.reset_index(drop = True)

# Treated group
# Fit a regression model to get the estimation of y given T=1 and X 
X1_high_treated = highDim_treated.drop(['Y'], axis = 1)
y_high_treated = highDim_treated['Y']

lr_high_treated = LinearRegression().fit(X1_high_treated, y_high_treated)

# Control group
# Fit a regression model to get the estimation of y given T=0 and X 
X1_high_control = highDim_control.drop(['Y'], axis = 1)
y_high_control = highDim_control['Y']

lr_high_control = LinearRegression().fit(X1_high_control, y_high_control)

##### Add estimations of control and treatment (i.e. m1 and m0) to the dataset

In [88]:
# Select all covariates and 'A' columns from full dataset
X_low_new = full_low_dim.drop(['Y','PS'], axis = 1)
X_high_new = full_high_dim.drop(['Y','PS'], axis = 1)

# For low dimensional dataset
m1_low= lr_low_treated.predict(X_low_new)
m0_low= lr_low_control.predict(X_low_new)


# For high dimensional dataset
m1_high= lr_high_treated.predict(X_high_new)
m0_high= lr_high_control.predict(X_high_new)

In [89]:
# join m1 and m0 to full_low_dim
full_low_dim['m1'] = pd.Series(m1_low, index = full_low_dim.index)
full_low_dim['m0'] = pd.Series(m0_low, index = full_low_dim.index)

# join m1 and m0 to full_high_dim
full_high_dim['m1'] = pd.Series(m1_high, index = full_high_dim.index)
full_high_dim['m0'] = pd.Series(m0_high, index = full_high_dim.index)

In [90]:
# Doubly Robust Estimation function
#full_data will contain Y, A, all covariates, Propensity scores, m1, and m0
def DRE(full_data):
    
    n = len(full_data.index)
    result1 = 0
    result2 = 0
    
    for i in range(n):
        result1 = result1 + (full_data['A'][i] * full_data['Y'][i] - (full_data['A'][i] - full_data['PS'][i])*full_data['m1'][i])/full_data['PS'][i]
        result2 = result2 + ((1-full_data['A'][i])* full_data['Y'][i] - (full_data['A'][i] - full_data['PS'][i])*full_data['m0'][i])/(1-full_data['PS'][i])
        
    ETA = 1/n*(result1-result2)
    
    return ETA

In [91]:
# Calculate ATE for low dimensional data
print(DRE(full_low_dim))

2.404023527246492


In [92]:
# Calculate ATE for high dimensional data
print(DRE(full_high_dim))

-3.0963460273733507


#### Doubly Robust Estimation Time

In [93]:
dre_time = time.time() - start_time_dre
print("run time: %s seconds" % (dre_time))

run time: 9.130182027816772 seconds


#### Accuracy of Doubly Robust Estimation

In [106]:
low_acc = abs(DRE(full_low_dim) - 2.5)/2.5 * 100
high_acc = (DRE(full_high_dim) - (-3))/(-3) * 100

# Summary

In [110]:
summary_table = pd.DataFrame(np.array([[l1_time, dre_time,low_acc, high_acc]]),
                   columns=['l1 running time', 'DRE running time', 'DRE lowDim accuracy', 'DRE highDim accuracy'])


In [111]:
summary_table

,l1 running time,DRE running time,DRE lowDim accuracy,DRE highDim accuracy
0,16.729212,9.130182,3.839059,3.211534
